# Business license parser

Parses raw business license data provided by city of Bozeman in May 2016, separating business and liquor licenses out into their own things.

TODO:
- Add address parsing to combine into in/outside city limits, full_address cols

In [17]:
import pandas as pd
import json
from datetime import datetime

pd.set_option('display.max_columns',100)

In [2]:
# Import license data from Excel file

all_licenses = pd.read_excel('raw-data/business_license_info_Eric_Chronicle_2002-2016__2016-05-02.xlsx')

In [3]:
# Import header column key from json

def get_json(file_path):
    """Imports json as dict"""
    with open(file_path) as f:
        return json.load(f)
    
col_key = get_json('raw-data/header-key.json')

In [4]:
# Rename columns
all_licenses.rename(columns=col_key, inplace=True)
all_licenses.columns

Index([u'biz_name', u'license_num', u'control_num', u'resp_dept_code',
       u'resp_dept_descr', u'class_code', u'class_descr', u'fiscal_year',
       u'license_status', u'code_descr', u'status_date', u'application_date',
       u'issue_date', u'licen_exp_date', u'renew_date', u'location_id',
       u'street_num', u'street_pre_qual', u'street_dir_code', u'street_name',
       u'street_suffix', u'street_post_qual', u'street_apt_num'],
      dtype='object')

In [5]:
# Parse dates
# Date input format, int in YYMMDD

def parse_date(value):
    d_format = '%y%m%d'
    if value == 0:
        pass
    else:
        string = str(value).zfill(6)
        parsed = pd.to_datetime(string, format=d_format)
        return parsed
    
date_cols = ['status_date','application_date','issue_date','licen_exp_date', 'renew_date']

date_corrected = all_licenses

for col in date_cols:
    date_corrected[col] = date_corrected[col].apply(parse_date)

In [8]:
all_licenses = date_corrected

In [10]:
all_licenses.head(10)

,biz_name,license_num,control_num,resp_dept_code,resp_dept_descr,class_code,class_descr,fiscal_year,license_status,code_descr,status_date,application_date,issue_date,licen_exp_date,renew_date,location_id,street_num,street_pre_qual,street_dir_code,street_name,street_suffix,street_post_qual,street_apt_num
0,... AND SEW ON,52085,25732,BL,BUSINESS LICENSE,SER 28,SERVICE - SEWING/ALTERATIONS,15,IN,INACTIVE,2015-12-16,2015-01-22,2015-08-02,2015-12-31,NaT,52760,601,NaN,NaN,NIKLES,DR,NaN,NaN
1,... AND SEW ON,53948,25732,BL,BUSINESS LICENSE,RET 38,RETAIL - OTHER,16,AC,ACTIVE,2016-03-02,2015-12-16,2016-03-02,2016-12-31,NaT,52760,601,NaN,NaN,NIKLES,DR,NaN,NaN
2,@ YOUR SERVICE,53148,26432,BL,BUSINESS LICENSE,SER 31,SERVICE - OTHER,15,RN,RENEWED,2016-01-13,2015-07-28,2015-10-12,2015-12-31,2016-01-13,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN
3,@ YOUR SERVICE,53148,26432,BL,BUSINESS LICENSE,SER 31,SERVICE - OTHER,16,AC,ACTIVE,2016-01-13,2016-01-13,2016-01-13,2016-12-31,NaT,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN
4,"""DO ME A PRETTY FAVOR PLEASE!""",19250,12322,BL,BUSINESS LICENSE,SER 31,SERVICE - OTHER,7,IN,INACTIVE,2008-04-09,2007-04-19,2007-04-19,2007-12-31,NaT,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN
5,"""I CAN DO IT"" GIFT BASKETS",8145,7676,BL,BUSINESS LICENSE,RET 38,RETAIL - OTHER,4,RN,RENEWED,2005-01-03,2004-03-02,2004-03-11,2004-12-31,2005-01-04,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN
6,"""I CAN DO IT"" GIFT BASKETS",8145,7676,BL,BUSINESS LICENSE,RET 38,RETAIL - OTHER,5,RN,RENEWED,2006-01-03,2005-01-04,2005-12-14,2005-12-31,2006-01-03,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN
7,"""I CAN DO IT"" GIFT BASKETS",15829,7676,BL,BUSINESS LICENSE,RET 38,RETAIL - OTHER,6,IN,INACTIVE,2009-09-22,2006-01-03,2006-04-27,2006-12-31,NaT,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN
8,"""I CAN DO IT"" GIFT BASKETS",8145,7676,BL,BUSINESS LICENSE,RET 38,RETAIL - OTHER,6,TR,TRANSFERRED,2006-06-14,2006-01-03,2006-01-03,2006-12-31,2006-06-14,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN
9,"""I LOVE TO CLEAN""",12439,9666,BL,BUSINESS LICENSE,CONT 15,CONTRACTORS - HANDY PERSONS,5,IN,INACTIVE,2006-02-22,2005-07-06,2005-12-13,2005-12-31,NaT,84770,100,NaN,NaN,OUT OF CITY LIMITS,ST,NaN,NaN


In [13]:
all_licenses['resp_dept_descr'].value_counts()

BUSINESS LICENSE    55650
LIQUOR LICENSE       3519
Name: resp_dept_descr, dtype: int64

In [24]:
biz_l = all_licenses[all_licenses['resp_dept_descr'] == 'BUSINESS LICENSE']
liq_l = all_licenses[all_licenses['resp_dept_descr'] == 'LIQUOR LICENSE']

In [54]:
def cond_get(array):
    if len(array) > 1:
        return array[1]
    else:
        pass

split = biz_l['class_descr'].apply(lambda text: text.split(' - '))
biz_l['class_desc_cat'] = split.apply(lambda x: x[0])
biz_l['class_desc_detail'] = split.apply(cond_get)

/Users/reporter/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/reporter/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [56]:
# Export to csv
biz_l.to_csv('working-data/bzn-biz-licenses.csv', index=False, encoding='utf-8')
liq_l.to_csv('working-data/bzn-liq-licenses.csv', index=False, encoding='utf-8')